### Michael Gainey

In [1]:
import re


# Assignment 2: Analyzing large datasets with Spark.


For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v $(pwd):/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [2]:
### Write your code here
from pyspark import SparkContext
sc = SparkContext()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/07 15:15:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
SparkContext?

In [4]:
print(sc.version)
print(sc.pythonVer)
print(sc.master)

3.2.0
3.9
local[*]


In [5]:
sc.getConf().getAll()

[('spark.app.id', 'local-1636298103524'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.driver.host', 'ce25afd91cf3'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1636298102477'),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '43129')]

### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [6]:
### Write your code here
book_43 = sc.textFile("data/43.txt")
print(str(type(book_43)) == "<class 'pyspark.rdd.RDD'>")

True


### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [7]:
### Write your code here
lines = book_43.count()
print(lines)

2935


### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [8]:
import re
### Write your code here
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()

test_line = "This is an example of that contains 234 and a dash-containing number"

print(clean_split_line(test_line))

['THIS', 'IS', 'AN', 'EXAMPLE', 'OF', 'THAT', 'CONTAINS', 'AND', 'A', 'DASH', 'CONTAINING', 'NUMBER']


### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [9]:
### Write your code here
amount_of_words = book_43.flatMap(clean_split_line).count()
amount_of_words

29116

### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [10]:
### Write your code here
def clean_split_line_lower(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.lower().split()

unique_word_count = book_43.flatMap(clean_split_line_lower).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
unique_word_count.filter(lambda x: x[1] == 1)
unique_word_count.count()

# idk how to do this one maybe

4296

### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [11]:
### Write your code here
book_43_counts = book_43.flatMap(clean_split_line_lower).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
book_43_counts.take(1)

[('project', 88)]

### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [12]:
### Write your code here
book_43_counts = book_43_counts.map(lambda x: (x[1], x[0])).sortByKey(ascending = False)
book_43_counts.take(20)

[(1807, 'the'),
 (1068, 'of'),
 (1043, 'and'),
 (726, 'to'),
 (686, 'a'),
 (646, 'i'),
 (485, 'in'),
 (471, 'was'),
 (392, 'that'),
 (384, 'he'),
 (378, 'it'),
 (312, 'you'),
 (308, 'my'),
 (301, 'with'),
 (285, 'his'),
 (244, 'had'),
 (203, 'as'),
 (202, 'for'),
 (195, 'this'),
 (193, 'but')]

### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [13]:
### Write your code here
import random
from sklearn.feature_extraction import _stop_words
stop_words = _stop_words.ENGLISH_STOP_WORDS
stop_words_subscript = list(stop_words)
print(random.sample(stop_words_subscript, 10))

['much', 'found', 'me', 'fire', 'many', 'whereupon', 'his', 'without', 'here', 'others']


### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [14]:
### Write your code here
new_book_43_counts = book_43_counts.map(lambda x: (x[1], x[0]))
new_book_43_counts_filtered = new_book_43_counts.filter(lambda x: x[0] not in stop_words_subscript)
new_book_43_counts_filtered.take(20)

[('s', 156),
 ('utterson', 131),
 ('said', 130),
 ('mr', 128),
 ('hyde', 105),
 ('jekyll', 105),
 ('gutenberg', 98),
 ('project', 88),
 ('man', 85),
 ('lawyer', 72),
 ('poole', 61),
 ('like', 59),
 ('sir', 59),
 ('tm', 57),
 ('door', 52),
 ('work', 49),
 ('dr', 49),
 ('life', 49),
 ('good', 45),
 ('hand', 45)]

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [15]:
### Write your code here
book_43_counts_filtered_count = new_book_43_counts_filtered.count()
book_43_counts_filtered_count

4034

### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [16]:
### Write your code here
# def process_RDD(file):
#     # reads file as a textRDD
#     rdd = sc.textFile(file)
# #     cleaned_split_rdd = rdd.flatMap(clean_split_line).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
# #     cleaned_split_filtered_rdd = cleaned_split_rdd.filter(lambda x: x[0] not in stop_words_subscript)
# #     return cleaned_split_filtered_rdd.count()
#     cleaned_split_rdd = rdd.flatMap(clean_split_line).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
#     cleaned_split_rdd_sorted = cleaned_split_rdd.map(lambda x: (x[1], x[0])).sortByKey(ascending = False)
#     cleaned_split_rdd_sorted_back = cleaned_split_rdd_sorted.map(lambda x: (x[1], x[0]))
#     filtered_rdd = cleaned_split_rdd_sorted_back.filter(lambda x: x[0] not in stop_words_subscript)
#     return filtered_rdd

In [17]:
def process_RDD(file):
    rdd = sc.textFile(file)
    cleaned_split_rdd = rdd.flatMap(clean_split_line_lower)
    rdd_count = cleaned_split_rdd.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
    filtered_rdd = rdd_count.filter(lambda x: x[0] not in stop_words_subscript)
    return filtered_rdd

### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [18]:
### Write your code here
book_84_counts_filtered = process_RDD('data/84.txt')
book_398_counts_filtered = process_RDD('data/398.txt')
book_3296_counts_filtered = process_RDD('data/3296.txt')

print(book_84_counts_filtered.count())
print(book_398_counts_filtered.count())
print(book_3296_counts_filtered.count())

7016
2421
7293


### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [19]:
### Write your code here
book_3296 = book_3296_counts_filtered.keys().collect()
book_398 = book_398_counts_filtered.keys().collect()

book_398_and_3296 = book_398_counts_filtered.filter(lambda x: x[0] in book_3296)
print(book_398_and_3296.count())
book_84_and_398 = book_84_counts_filtered.filter(lambda x: x[0] in book_398)
print(book_84_and_398.count())

1790
1691


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

In [20]:
book_84_and_3296 = book_84_counts_filtered.filter(lambda x: x[0] in book_3296)
print(book_84_and_3296.count())

3608


#### Write your answer here

I don't think that counting the number of shared words is a good idea as a distance metric because book 398 and 3296 have less shared words than book 84 and 3296. Even though book 398 and 3296 both pertain to religion, the amount of shared words is less than 84 and 3296. 

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


In [21]:
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")

The Euclidean distance between A and B is: 3.1622776601683795
The Euclidean distance between A and C is: 14.0
The Euclidean distance between B and C is: 16.431676725154983


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [22]:
### Write your code here
"""
Books
43.txt	The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson
84.txt	Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley
398.txt	The First Book of Adam and Eve by Rutherford Hayes Platt
3296.txt	The Confessions of St. Augustine by Bishop of Hippo Saint Augustine
"""
import numpy as np

# find words that are common to all 4 documents after stop-word filtering
book_43 = process_RDD('data/43.txt')
book_84 = process_RDD('data/84.txt')
book_398 = process_RDD('data/398.txt')
book_3296 = process_RDD('data/3296.txt')

# common_books_43_and_84 = book_43.union(book_84).reduceByKey(lambda x,y: x+y)
# common_books_398_and_3296 = book_398.union(book_3296).reduceByKey(lambda x,y: x+y)
# common_books = common_books_43_and_84.union(common_books_398_and_3296).reduceByKey(lambda x,y: x+y)
# joined_common_books = common_books.intersection(common_books_43_and_84)
common_books_43_and_84 = book_43.join(book_84)
common_books_398_and_3296 = book_398.join(book_3296)
joined_common_books = common_books_43_and_84.join(common_books_398_and_3296)
joined_common_books_count = joined_common_books.count()
joined_common_books = joined_common_books.keys().collect()
# print(joined_common_books_count)
# joined_common_books is common words that appear in all 4 docs

# filter each book by joined_common_books
count_43 = book_43.filter(lambda x: x[0] in joined_common_books).sortByKey(ascending = False)
# remove the word and only show numbers
count_43 = count_43.map(lambda x: (x[1], x[0])).keys().collect()

count_84 = book_84.filter(lambda x: x[0] in joined_common_books).sortByKey(ascending = False)
count_84 = count_84.map(lambda x: (x[1], x[0])).keys().collect()

count_398 = book_398.filter(lambda x: x[0] in joined_common_books).sortByKey(ascending = False)
count_398 = count_398.map(lambda x: (x[1], x[0])).keys().collect()

count_3296 = book_3296.filter(lambda x: x[0] in joined_common_books).sortByKey(ascending = False)
count_3296 = count_3296.map(lambda x: (x[1], x[0])).keys().collect()

# get count for columns
column_count = len(count_43)
# print(len(count_43))

# np.zeros([rows,columns])
# create matrix
counts_matrix = np.zeros([4, column_count])

# example for A == counts_matrix[0, :] = [4, 9, 6, 8] 
counts_matrix[0, :] = count_43
counts_matrix[1, :] = count_84
counts_matrix[2, :] = count_398
counts_matrix[3, :] = count_3296

counts_matrix

array([[ 1.,  3.,  5., ...,  2.,  3.,  1.],
       [ 1.,  9., 30., ...,  4.,  9.,  1.],
       [ 1.,  2.,  1., ..., 16.,  5.,  1.],
       [ 1.,  4., 15., ...,  3., 36., 16.]])

### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?
<!--  -->

In [23]:
### Write your code here
from scipy.spatial.distance import euclidean
print(f"The Euclidean distance between book_398 and book_3296 is: {euclidean(counts_matrix[2], counts_matrix[3])}")
print(f"The Euclidean distance between book_84 and book_398 is: {euclidean(counts_matrix[1], counts_matrix[2])}")

The Euclidean distance between book_398 and book_3296 is: 1156.6628722320086
The Euclidean distance between book_84 and book_398 is: 751.6688100486809


#### Write your justification here

Using Euclidean distance is not helpful for determining relatedness across documents because even though book_398 and book_3296 are related by religion their distance is still greater than book_84 and book_398. 

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

#### Write your answer here


## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [24]:
### Write your code here
from pyspark.sql import SparkSession
session = SparkSession(sc)

flight_info = session.read.csv('data/flight_info.csv', header = True)
print(flight_info.count())
# flight_info.first()

450017


### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [25]:
### Write your code here
carriers = flight_info.select('UniqueCarrier')
carriers.show()

+-------------+
|UniqueCarrier|
+-------------+
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
+-------------+
only showing top 20 rows



### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [26]:
### Write your code here
# get cols
flight_df = flight_info.select('UniqueCarrier', 'CRSDepTime', 'DepTime')
# flight_df.show()
# filter for delays
flight_df = flight_df.filter(flight_info['DepTime'] > flight_info['CRSDepTime'])
# flight_df.show()
# group by code
flight_df = flight_df.groupby('UniqueCarrier').count()
# flight_df.show()
# sort by count descending
flight_df = flight_df.sort('count', ascending = False)
flight_df.show()

+-------------+-----+
|UniqueCarrier|count|
+-------------+-----+
|           WN|54557|
|           DL|28962|
|           AA|26291|
|           UA|19594|
|           OO|17924|
|           EV|12340|
|           B6|10406|
|           AS| 4966|
|           NK| 4435|
|           F9| 3181|
|           VX| 2871|
|           HA| 2265|
+-------------+-----+



### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [27]:
### Write your code here
airlines_info = session.read.csv('data/airlines.csv')
print(airlines_info)
airlines_info.show(5)

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string]
+---+--------------------+---+----+---+-------+--------------+---+
|_c0|                 _c1|_c2| _c3|_c4|    _c5|           _c6|_c7|
+---+--------------------+---+----+---+-------+--------------+---+
|  1|      Private flight| \N|   -|N/A|   null|          null|  Y|
|  2|         135 Airways| \N|null|GNL|GENERAL| United States|  N|
|  3|       1Time Airline| \N|  1T|RNX|NEXTIME|  South Africa|  Y|
|  4|2 Sqn No 1 Elemen...| \N|null|WYT|   null|United Kingdom|  N|
|  5|     213 Flight Unit| \N|null|TFU|   null|        Russia|  N|
+---+--------------------+---+----+---+-------+--------------+---+
only showing top 5 rows



In [28]:
delayed_df = flight_info.join(airlines_info, flight_info['UniqueCarrier'] == airlines_info['_c3'])
delayed_df = delayed_df.select('UniqueCarrier', '_c1', 'CRSDepTime', 'DepTime')
delayed_df = delayed_df.filter(flight_info['DepTime'] > flight_info['CRSDepTime'])
delayed_df = delayed_df.groupby('UniqueCarrier', '_c1').count()
delayed_df = delayed_df.sort('count', ascending = False)

delayed_df.show()


+-------------+--------------------+-----+
|UniqueCarrier|                 _c1|count|
+-------------+--------------------+-----+
|           WN|  Southwest Airlines|54557|
|           DL|     Delta Air Lines|28962|
|           AA|   American Airlines|26291|
|           UA|     United Airlines|19594|
|           OO|             SkyWest|17924|
|           EV|Atlantic Southeas...|12340|
|           B6|     JetBlue Airways|10406|
|           AS|     Alaska Airlines| 4966|
|           NK|     Spirit Airlines| 4435|
|           F9|   Frontier Airlines| 3181|
|           VX|      Virgin America| 2871|
|           HA|   Hawaiian Airlines| 2265|
+-------------+--------------------+-----+



### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [29]:
### Write your code here
daily_delay = flight_info.select('UniqueCarrier', 'CRSDepTime', 'DepTime', 'DayOfWeek')
daily_delay = daily_delay.filter(flight_info['DepTime'] > flight_info['CRSDepTime'])
daily_delay = daily_delay.groupby('UniqueCarrier', 'DayOfWeek').count()
daily_delay = daily_delay.sort(['UniqueCarrier', 'DayOfWeek'])

daily_delay.show()

+-------------+---------+-----+
|UniqueCarrier|DayOfWeek|count|
+-------------+---------+-----+
|           AA|        1| 5117|
|           AA|        2| 3688|
|           AA|        3| 2941|
|           AA|        4| 3575|
|           AA|        5| 3525|
|           AA|        6| 2800|
|           AA|        7| 4645|
|           AS|        1|  872|
|           AS|        2|  669|
|           AS|        3|  618|
|           AS|        4|  728|
|           AS|        5|  630|
|           AS|        6|  517|
|           AS|        7|  932|
|           B6|        1| 1892|
|           B6|        2| 1787|
|           B6|        3| 1253|
|           B6|        4| 1254|
|           B6|        5| 1309|
|           B6|        6| 1027|
+-------------+---------+-----+
only showing top 20 rows



### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [30]:
%%time
### Write your code here
# get total of flights per airline
flights = flight_info.select('UniqueCarrier', 'DayOfWeek')
flights = flights.groupby('UniqueCarrier', 'DayOfWeek').count()
flights = flights.sort('UniqueCarrier', 'DayOfWeek')
# flights has count of flights per day of the week
# flights.show()

# get delays
delays = flight_info.select('UniqueCarrier', 'DayOfWeek', 'CRSDepTime', 'DepTime')
delays = delays.filter(flight_info['DepTime'] > flight_info['CRSDepTime'])
delays = delays.groupby('UniqueCarrier', 'DayOfWeek').count()
delays = delays.sort(['UniqueCarrier', 'DayOfWeek'])
# delays.show()

daily_delays = delays.join(flights, ['UniqueCarrier', 'DayOfWeek'])
daily_delays = daily_delays.sort('UniqueCarrier', 'DayOfWeek')
daily_delays = daily_delays.withColumn('NormalizedCounts', delays['count']/flights['count'])
daily_delays = daily_delays.drop('count', 'count')
daily_delays.show()


+-------------+---------+-------------------+
|UniqueCarrier|DayOfWeek|   NormalizedCounts|
+-------------+---------+-------------------+
|           AA|        1| 0.4206329634196465|
|           AA|        2|0.31950099627479855|
|           AA|        3|0.30285243538255585|
|           AA|        4|0.36133009904992924|
|           AA|        5|0.35448511665325827|
|           AA|        6| 0.3415883859948762|
|           AA|        7| 0.3977564651481418|
|           AS|        1| 0.3553382233088835|
|           AS|        2|0.29099608525445847|
|           AS|        3| 0.3333333333333333|
|           AS|        4| 0.3731419784725782|
|           AS|        5|0.32142857142857145|
|           AS|        6| 0.2875417130144605|
|           AS|        7|0.38914405010438413|
|           B6|        1|0.45811138014527847|
|           B6|        2|0.44899497487437184|
|           B6|        3|0.39601769911504425|
|           B6|        4|0.38278388278388276|
|           B6|        5|0.4005507

### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [31]:
### Write your code here
"""
CPU times: user 17.6 ms, sys: 7.91 ms, total: 25.5 ms
Wall time: 1.05 s
CPU times: user 17.3 ms, sys: 4.41 ms, total: 21.7 ms
Wall time: 834 ms
CPU times: user 15 ms, sys: 10.5 ms, total: 25.5 ms
Wall time: 795 ms
CPU times: user 11.6 ms, sys: 8.11 ms, total: 19.7 ms
Wall time: 791 ms
CPU times: user 11.7 ms, sys: 8.18 ms, total: 19.9 ms
Wall time: 721 ms
CPU times: user 10.6 ms, sys: 14.6 ms, total: 25.2 ms
Wall time: 738 ms
CPU times: user 21.7 ms, sys: 0 ns, total: 21.7 ms
Wall time: 733 ms
CPU times: user 11.1 ms, sys: 10.9 ms, total: 21.9 ms
Wall time: 719 ms
CPU times: user 14.8 ms, sys: 3.94 ms, total: 18.7 ms
Wall time: 724 ms
CPU times: user 3.67 ms, sys: 17 ms, total: 20.7 ms
Wall time: 720 ms
"""

time = [1050, 834, 795, 791, 721, 738, 733, 719, 724, 720]
sum = 0
for i in time:
    sum = sum + i
    
av_time = sum / len(time)
print(str(av_time) + 'ms')


782.5ms


### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [32]:
### Write your code here
flights.cache()
delays.cache()
daily_delays.cache()

DataFrame[UniqueCarrier: string, DayOfWeek: string, NormalizedCounts: double]

In [49]:
%%time
### Write your code here
# get total of flights per airline
flights = flight_info.select('UniqueCarrier', 'DayOfWeek')
flights = flights.groupby('UniqueCarrier', 'DayOfWeek').count()
flights = flights.sort('UniqueCarrier', 'DayOfWeek')
# flights has count of flights per day of the week
# flights.show()

# get delays
delays = flight_info.select('UniqueCarrier', 'DayOfWeek', 'CRSDepTime', 'DepTime')
delays = delays.filter(flight_info['DepTime'] > flight_info['CRSDepTime'])
delays = delays.groupby('UniqueCarrier', 'DayOfWeek').count()
delays = delays.sort(['UniqueCarrier', 'DayOfWeek'])
# delays.show()

daily_delays = delays.join(flights, ['UniqueCarrier', 'DayOfWeek'])
daily_delays = daily_delays.sort('UniqueCarrier', 'DayOfWeek')
daily_delays = daily_delays.withColumn('NormalizedCounts', delays['count']/flights['count'])
daily_delays = daily_delays.drop('count', 'count')
daily_delays.show()


+-------------+---------+-------------------+
|UniqueCarrier|DayOfWeek|   NormalizedCounts|
+-------------+---------+-------------------+
|           AA|        1| 0.4206329634196465|
|           AA|        2|0.31950099627479855|
|           AA|        3|0.30285243538255585|
|           AA|        4|0.36133009904992924|
|           AA|        5|0.35448511665325827|
|           AA|        6| 0.3415883859948762|
|           AA|        7| 0.3977564651481418|
|           AS|        1| 0.3553382233088835|
|           AS|        2|0.29099608525445847|
|           AS|        3| 0.3333333333333333|
|           AS|        4| 0.3731419784725782|
|           AS|        5|0.32142857142857145|
|           AS|        6| 0.2875417130144605|
|           AS|        7|0.38914405010438413|
|           B6|        1|0.45811138014527847|
|           B6|        2|0.44899497487437184|
|           B6|        3|0.39601769911504425|
|           B6|        4|0.38278388278388276|
|           B6|        5|0.4005507

In [37]:
"""
CPU times: user 3.36 ms, sys: 16.9 ms, total: 20.3 ms
Wall time: 149 ms
CPU times: user 3.2 ms, sys: 16.3 ms, total: 19.5 ms
Wall time: 134 ms
CPU times: user 9.22 ms, sys: 9.77 ms, total: 19 ms
Wall time: 138 ms
CPU times: user 11.5 ms, sys: 7.75 ms, total: 19.3 ms
Wall time: 138 ms
CPU times: user 15.6 ms, sys: 3.08 ms, total: 18.6 ms
Wall time: 152 ms
CPU times: user 11.2 ms, sys: 11.1 ms, total: 22.3 ms
Wall time: 152 ms
CPU times: user 4.36 ms, sys: 14 ms, total: 18.3 ms
Wall time: 127 ms
CPU times: user 9.44 ms, sys: 9.98 ms, total: 19.4 ms
Wall time: 129 ms
CPU times: user 14.2 ms, sys: 5.74 ms, total: 20 ms
Wall time: 131 ms
CPU times: user 20.3 ms, sys: 0 ns, total: 20.3 ms
Wall time: 135 ms
"""

new_time = [149, 134, 138, 138, 152, 152, 127, 129, 131, 135]
sum = 0
for i in new_time:
    sum = sum + i
    
av_time = sum / len(new_time)
print(str(av_time) + 'ms')

138.5ms


### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [68]:
### Write your code here
# find departure delay
departure_delay = flight_info.select('DepTime', 'CRSDepTime')
departure_delay = departure_delay.withColumn('DepartureDelay', flight_info['DepTime'] - flight_info['CRSDepTime'])
departure_delay = departure_delay.drop('DepTime', 'CRSDepTime')
# departure_delay.show()
# departure_delay = departure_delay.collect()
print(departure_delay.show())


# find arrival delay
arrival_delay = flight_info.select('ArrTime', 'CRSArrTime')
arrival_delay = arrival_delay.withColumn('ArrivalDelay', flight_info['ArrTime'] - flight_info['CRSArrTime'])
arrival_delay = arrival_delay.drop('ArrTime', 'CRSArrTime')
# arrival_delay.show()

delays = departure_delay.join(arrival_delay)
# delays = arrival_delay.join(departure_delay)
delays.show()
# import sklearn.linear_model as skl_lm
# from sklearn.metrics import mean_squared_error, r2_score



+--------------+
|DepartureDelay|
+--------------+
|          -3.0|
|          -5.0|
|          -8.0|
|          37.0|
|          13.0|
|          17.0|
|          -3.0|
|           0.0|
|          -3.0|
|          -1.0|
|          -6.0|
|          -5.0|
|          -8.0|
|          -5.0|
|          -4.0|
|           0.0|
|         -49.0|
|           7.0|
|           9.0|
|          -1.0|
+--------------+
only showing top 20 rows

None
+--------------+------------+
|DepartureDelay|ArrivalDelay|
+--------------+------------+
|          -3.0|       -14.0|
|          -3.0|       -35.0|
|          -3.0|       -30.0|
|          -3.0|        73.0|
|          -3.0|         2.0|
|          -3.0|        65.0|
|          -3.0|        51.0|
|          -3.0|        48.0|
|          -3.0|        50.0|
|          -3.0|        46.0|
|          -3.0|       -14.0|
|          -3.0|       -33.0|
|          -3.0|       -28.0|
|          -3.0|       -20.0|
|          -3.0|       -20.0|
|          -3.0|     